In [ ]:
"""
BOPTEST HVAC PPO 训练 - 强化版
- 更新动作空间（物理范围内的离散组合，强制 heat<cool）
- 丰富观测（天气/辐照/CO2/时间/上一步动作）
- 奖励=舒适度二次惩罚 + 能耗惩罚 + 动作平滑
- GAE 末状态 bootstrap，超参数更稳（更小 lr，稍高熵系数）
"""

import os
import sys
import requests
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import time
import json
import csv
from datetime import datetime
from collections import Counter
import matplotlib.pyplot as plt
from IPython.display import clear_output

# =====================================================================
# 配置参数
# =====================================================================
BOPTEST_URL = os.getenv("BOPTEST_URL", "http://localhost:80")  # 与 DQN 保持一致
TESTCASE_NAME = "bestest_air"

# 训练参数（拉长训练，更稳探索）
# 给你 10 小时预算，直接长跑 500 ep
TOTAL_EPISODES = 500
EVAL_FREQUENCY = 20
STEPS_PER_EPISODE = 96  # 1 day (96 steps = 1 day, 15min per step)
PLOT_SAVE_DIR = "training_output"
DATA_SAVE_DIR = "training_data"  # 数据保存目录

# PPO 超参数（稳健收敛，可在此基础上小幅微调）
PPO_EPOCHS = 5
PPO_CLIP = 0.2
GAMMA = 0.99
GAE_LAMBDA = 0.95
# 已改为分别设置actor和critic学习率
LEARNING_RATE_ACTOR = 1e-4   # Actor学习率：小一点，更稳定
LEARNING_RATE_CRITIC = 5e-4  # Critic学习率：大一点，先学好价值估计
ENTROPY_COEF = 0.05           # 略高探索，长跑更稳
VALUE_COEF = 0.5
MAX_GRAD_NORM = 0.5
MINI_BATCH_SIZE = 256         # 适当放大 batch，梯度更平滑

# =====================================================================
# 动作空间（与 DQN 保持一致）
# =====================================================================
# 范围来源：
#  - fcu_oveFan_u: 0~1
#  - fcu_oveTSup_u: 285.15~313.15 K
#  - con_oveTSetHea_u / con_oveTSetCoo_u: 278.15~308.15 K
# 强制 cool >= heat + 1.5K 以保证死区
# 3 (fan) x 2 (supply) x 2 (heat) x 2 (cool) = 24 actions
FAN_LEVELS = [0.3, 0.6, 0.9]
SUPPLY_TEMP_LEVELS = [288.15, 296.15]
HEAT_SETPOINT_LEVELS = [294.15, 296.15]
COOL_SETPOINT_LEVELS = [299.15, 301.15]

NUM_ACTIONS = (
    len(FAN_LEVELS)
    * len(SUPPLY_TEMP_LEVELS)
    * len(HEAT_SETPOINT_LEVELS)
    * len(COOL_SETPOINT_LEVELS)
)

# =====================================================================
# 工具函数
# =====================================================================

def log(msg, level="INFO"):
    timestamp = datetime.now().strftime("%H:%M:%S")
    print(f"[{timestamp}] {level:5s} | {msg}", flush=True)

def safe_json(resp, context: str = ""):
    try:
        return resp.json()
    except Exception as e:
        log(
            f"JSON 解析失败({context}): {type(e).__name__}: {e}. status={resp.status_code}, text={resp.text[:300]}",
            "ERROR",
        )
        return None

def K2C(k):
    return k - 273.15

def action_to_values(action_idx):
    """将索引解码为各控制量并做安全裁剪"""
    cool_idx = action_idx % len(COOL_SETPOINT_LEVELS)
    remaining = action_idx // len(COOL_SETPOINT_LEVELS)
    heat_idx = remaining % len(HEAT_SETPOINT_LEVELS)
    remaining //= len(HEAT_SETPOINT_LEVELS)
    supply_idx = remaining % len(SUPPLY_TEMP_LEVELS)
    fan_idx = remaining // len(SUPPLY_TEMP_LEVELS)

    v = {
        'fan': FAN_LEVELS[fan_idx],
        'supply_temp': SUPPLY_TEMP_LEVELS[supply_idx],
        'heat_setpoint': HEAT_SETPOINT_LEVELS[heat_idx],
        'cool_setpoint': COOL_SETPOINT_LEVELS[cool_idx],
    }

    # 强制 cool > heat + 1.5K
    if v['cool_setpoint'] <= v['heat_setpoint'] + 1.5:
        v['cool_setpoint'] = v['heat_setpoint'] + 1.5

    # 物理范围裁剪
    v['fan'] = float(np.clip(v['fan'], 0.0, 1.0))
    v['supply_temp'] = float(np.clip(v['supply_temp'], 285.15, 313.15))
    v['heat_setpoint'] = float(np.clip(v['heat_setpoint'], 278.15, 308.15))
    v['cool_setpoint'] = float(np.clip(v['cool_setpoint'], 278.15, 308.15))
    return v

def action_to_string(action_idx):
    v = action_to_values(action_idx)
    return (
        f"Fan={v['fan']:.1f}|供风={K2C(v['supply_temp']):.0f}°C|"
        f"热设={K2C(v['heat_setpoint']):.1f}°C|冷设={K2C(v['cool_setpoint']):.1f}°C"
    )

# =====================================================================
# 环境
# =====================================================================

class BOPTESTEnv:
    def __init__(self):
        self.url = BOPTEST_URL
        self.testid = None
        self.obs_dim = 12  # 扩展观测
        self.action_dim = NUM_ACTIONS
        self.prev_action_norm = 0.0

    def _select_testcase(self):
        log(f"选择测试案例: {TESTCASE_NAME}")
        try:
            resp = requests.post(
                f"{self.url}/testcases/{TESTCASE_NAME}/select", timeout=30
            )
            if resp.status_code == 200:
                data = safe_json(resp, "select testcase")
                if data:
                    self.testid = data.get('testid')
                    if self.testid:
                        log(f"✅ Test ID: {self.testid}")
                        time.sleep(3)
                        return True
                    else:
                        log("❌ 未返回 testid", "ERROR")
            else:
                log(
                    f"❌ 选择失败，状态码={resp.status_code}, 响应={resp.text[:200]}",
                    "ERROR",
                )
        except Exception as e:
            log(f"❌ 选择失败: {e}", "ERROR")
        return False

    def reset(self):
        # 若无法选择测试案例，先打印可用案例便于诊断
        if not self.testid:
            try:
                list_resp = requests.get(f"{self.url}/testcases", timeout=10)
                if list_resp.status_code == 200:
                    data = safe_json(list_resp, "list testcases")
                    if data is not None:
                        log(f"可用测试案例: {list(data)}", "DEBUG")
                else:
                    log(
                        f"获取测试案例失败: {list_resp.status_code}, text={list_resp.text[:200]}",
                        "WARN",
                    )
            except Exception as e:
                log(f"获取测试案例列表失败: {e}", "WARN")

        if not self.testid and not self._select_testcase():
            raise Exception("无法选择测试案例")

        requests.put(
            f"{self.url}/initialize/{self.testid}",
            json={"start_time": 0, "warmup_period": 0},
            timeout=30,
        )
        requests.put(
            f"{self.url}/step/{self.testid}", json={"step": 900}, timeout=10
        )

        resp = requests.post(
            f"{self.url}/advance/{self.testid}", json={}, timeout=30
        )
        payload = resp.json().get('payload', {})
        self.prev_action_norm = 0.0
        return self._get_obs(payload)

    def step(self, action_idx):
        action = action_to_values(action_idx)

        control = {
            'fcu_oveFan_u': action['fan'],
            'fcu_oveFan_activate': 1,
            'fcu_oveTSup_u': action['supply_temp'],
            'fcu_oveTSup_activate': 1,
            'con_oveTSetHea_u': action['heat_setpoint'],
            'con_oveTSetHea_activate': 1,
            'con_oveTSetCoo_u': action['cool_setpoint'],
            'con_oveTSetCoo_activate': 1,
        }

        resp = requests.post(
            f"{self.url}/advance/{self.testid}", json=control, timeout=30
        )
        payload = resp.json().get('payload', {}) or {}

        obs = self._get_obs(payload)
        reward, reward_detail = self._calc_reward(obs)

        sim_time = payload.get('time', 0)
        done = sim_time >= STEPS_PER_EPISODE * 900

        info = {
            'sim_time': sim_time,
            'room_temp': K2C(obs[0]),
            'outdoor_temp': K2C(obs[1]),
            'reward_detail': reward_detail,
        }

        self.prev_action_norm = action_idx / max(1, NUM_ACTIONS - 1)
        return obs, reward, done, info

    def _safe_get(self, payload, key, default):
        v = payload.get(key, default)
        if v is None:
            return default
        return v

    def _get_obs(self, payload):
        if not payload:
            return np.zeros(self.obs_dim, dtype=np.float32)

        sim_time = payload.get('time', 0)
        hour = (sim_time % 86400) / 3600.0
        day = (sim_time // 86400) % 365

        room_temp = self._safe_get(payload, 'zon_reaTRooAir_y', 293.15)
        outdoor_temp = self._safe_get(payload, 'zon_weaSta_reaWeaTDryBul_y', 283.15)
        rel_hum = self._safe_get(payload, 'zon_weaSta_reaWeaRelHum_y', 0.5)
        solar = self._safe_get(payload, 'zon_weaSta_reaWeaHGloHor_y', 0.0)
        wind = self._safe_get(payload, 'zon_weaSta_reaWeaWinSpe_y', 0.0)
        co2 = self._safe_get(payload, 'zon_reaCO2RooAir_y', 600.0)
        p_heating = self._safe_get(payload, 'fcu_reaPHea_y', 0.0)
        p_cooling = self._safe_get(payload, 'fcu_reaPCoo_y', 0.0)
        p_fan = self._safe_get(payload, 'fcu_reaPFan_y', 0.0)

        obs = np.array(
            [
                room_temp,
                outdoor_temp,
                rel_hum,
                solar / 1000.0,  # kW/m2 量级
                wind,
                co2 / 1000.0,  # kppm 量级
                p_heating / 1000.0,  # kW
                p_cooling / 1000.0,  # kW
                p_fan / 1000.0,  # kW
                np.sin(2 * np.pi * hour / 24),
                np.cos(2 * np.pi * hour / 24),
                self.prev_action_norm,
            ],
            dtype=np.float32,
        )

        return obs

    def _calc_reward(self, obs):
        room_temp_k = obs[0]
        room_temp_c = K2C(room_temp_k)
        p_h, p_c, p_f = obs[6], obs[7], obs[8]

        # 【与 DQN 一致】Reward：舒适区内统一奖励，区外基于距离边界的惩罚（距离越大惩罚越大）
        COMFORT_LOW, COMFORT_HIGH = 20.0, 24.0
        if COMFORT_LOW <= room_temp_c <= COMFORT_HIGH:
            comfort = 3.0
        else:
            if room_temp_c < COMFORT_LOW:
                temp_dev_from_boundary = COMFORT_LOW - room_temp_c
            else:  # room_temp_c > COMFORT_HIGH
                temp_dev_from_boundary = room_temp_c - COMFORT_HIGH
            comfort = -3.0 * (temp_dev_from_boundary ** 2)  # 降低惩罚斜率，便于爬坡
        
        # 极端温度额外硬惩罚（减弱以便可恢复）
        if room_temp_c < 18.0 or room_temp_c > 26.0:
            comfort -= 15.0

        # 能耗惩罚：优先舒适，权重保持温和
        if 20.0 <= room_temp_c <= 24.0:
            energy = -0.15 * (p_h + p_c) - 0.03 * p_f
        else:
            energy = -0.05 * (p_h + p_c) - 0.01 * p_f
        
        smooth = -0.01 * abs(obs[-1] - self.prev_action_norm)

        reward = comfort + energy + smooth
        # 奖励裁剪，避免梯度被极端值主导
        reward = float(np.clip(reward, -50.0, 5.0))

        detail = {
            'comfort': comfort,
            'energy': energy,
            'smooth': smooth,
            'room_temp_c': room_temp_c,
            'in_comfort_zone': 20.0 <= room_temp_c <= 24.0,
        }
        return reward, detail

    def stop(self):
        if self.testid:
            try:
                requests.put(f"{self.url}/stop/{self.testid}", timeout=10)
                self.testid = None
            except Exception as e:
                log(f"停止失败: {e}", "WARN")

# =====================================================================
# PPO 网络
# =====================================================================

class ActorCritic(nn.Module):
    def __init__(self, state_dim, action_dim, hidden=128):
        super().__init__()

        self.actor = nn.Sequential(
            nn.Linear(state_dim, hidden),
            nn.Tanh(),
            nn.Linear(hidden, hidden),
            nn.Tanh(),
            nn.Linear(hidden, action_dim),
        )

        self.critic = nn.Sequential(
            nn.Linear(state_dim, hidden),
            nn.Tanh(),
            nn.Linear(hidden, hidden),
            nn.Tanh(),
            nn.Linear(hidden, 1),
        )

        # 正交初始化：前层用较大 gain，最后一层小 gain 保守输出
        for m in self.modules():
            if isinstance(m, nn.Linear):
                gain = np.sqrt(2)
                if m.out_features == action_dim or m.out_features == 1:
                    gain = 0.01
                nn.init.orthogonal_(m.weight, gain=gain)
                nn.init.zeros_(m.bias)

    def get_action(self, state, deterministic=False):
        logits = self.actor(state)
        probs = torch.softmax(logits, dim=-1)
        dist = torch.distributions.Categorical(probs)

        if deterministic:
            action = probs.argmax(dim=-1)
        else:
            action = dist.sample()

        return action, dist.log_prob(action), dist.entropy(), probs

    def get_value(self, state):
        return self.critic(state).squeeze(-1)

    def evaluate_action(self, state, action):
        logits = self.actor(state)
        probs = torch.softmax(logits, dim=-1)
        dist = torch.distributions.Categorical(probs)

        log_prob = dist.log_prob(action)
        entropy = dist.entropy()
        value = self.critic(state).squeeze(-1)

        return log_prob, entropy, value

# =====================================================================
# PPO 智能体
# =====================================================================

class PPOAgent:
    def __init__(self, state_dim, action_dim):
        self.model = ActorCritic(state_dim, action_dim)
        # 分别设置actor和critic的学习率
        self.optimizer = optim.Adam([
            {'params': self.model.actor.parameters(), 'lr': LEARNING_RATE_ACTOR},
            {'params': self.model.critic.parameters(), 'lr': LEARNING_RATE_CRITIC}
        ])

        self.state_dim = state_dim
        self.action_dim = action_dim

        log(f"PPO Agent 初始化: {action_dim} actions")
        log(f"  Actor学习率: {LEARNING_RATE_ACTOR}")
        log(f"  Critic学习率: {LEARNING_RATE_CRITIC}")
        log(f"  熵系数: {ENTROPY_COEF}")
        log(f"  PPO clip: {PPO_CLIP}")

    def select_action(self, state, deterministic=False):
        with torch.no_grad():
            state_t = torch.FloatTensor(state).unsqueeze(0)
            action, log_prob, entropy, probs = self.model.get_action(
                state_t, deterministic
            )
            value = self.model.get_value(state_t)
            return action.item(), log_prob.item(), value.item(), probs[0].numpy()

    def update(self, rollout):
        states = torch.FloatTensor(np.array(rollout['states']))
        actions = torch.LongTensor(rollout['actions'])
        old_log_probs = torch.FloatTensor(rollout['log_probs'])
        rewards = rollout['rewards']
        dones = rollout['dones']
        values = rollout['values']
        last_value = rollout['last_value']

        # 计算GAE，末步使用 bootstrap value
        advantages = []
        returns = []
        gae = 0

        for t in reversed(range(len(rewards))):
            next_value = last_value if t == len(rewards) - 1 else values[t + 1]
            delta = rewards[t] + GAMMA * next_value * (1 - dones[t]) - values[t]
            gae = delta + GAMMA * GAE_LAMBDA * (1 - dones[t]) * gae
            advantages.insert(0, gae)
            returns.insert(0, gae + values[t])

        advantages = torch.FloatTensor(advantages)
        returns = torch.FloatTensor(returns)

        advantages = (advantages - advantages.mean()) / (advantages.std() + 1e-8)

        total_loss = total_policy = total_value = total_entropy = 0.0
        num_updates = 0
        dataset_size = len(states)

        for _ in range(PPO_EPOCHS):
            indices = np.random.permutation(dataset_size)

            for start in range(0, dataset_size, MINI_BATCH_SIZE):
                end = min(start + MINI_BATCH_SIZE, dataset_size)
                idx = indices[start:end]

                new_log_probs, entropy, new_values = self.model.evaluate_action(
                    states[idx], actions[idx]
                )

                ratio = torch.exp(new_log_probs - old_log_probs[idx])
                surr1 = ratio * advantages[idx]
                surr2 = torch.clamp(ratio, 1 - PPO_CLIP, 1 + PPO_CLIP) * advantages[idx]
                policy_loss = -torch.min(surr1, surr2).mean()

                value_loss = nn.functional.mse_loss(new_values, returns[idx])
                entropy_loss = entropy.mean()

                loss = policy_loss + VALUE_COEF * value_loss - ENTROPY_COEF * entropy_loss

                self.optimizer.zero_grad()
                loss.backward()
                nn.utils.clip_grad_norm_(self.model.parameters(), MAX_GRAD_NORM)
                self.optimizer.step()

                total_loss += loss.item()
                total_policy += policy_loss.item()
                total_value += value_loss.item()
                total_entropy += entropy_loss.item()
                num_updates += 1

        return {
            'loss': total_loss / num_updates,
            'policy_loss': total_policy / num_updates,
            'value_loss': total_value / num_updates,
            'entropy': total_entropy / num_updates,
        }

    def save(self, path):
        torch.save(self.model.state_dict(), path)
        log(f"💾 模型已保存: {path}")

# =====================================================================
# 训练监控
# =====================================================================

class Monitor:
    def __init__(self, save_dir="training_data"):
        self.save_dir = save_dir
        os.makedirs(save_dir, exist_ok=True)
        
        self.train_rewards = []
        self.train_temps = []
        self.train_comfort_ratios = []  # 舒适区时间比例
        self.train_energy_consumption = []  # 能耗
        self.eval_rewards = []
        self.eval_episodes = []
        self.entropies = []
        
        # 详细数据记录
        self.episode_data = []  # 每episode的详细数据
        
        # 当前episode的轨迹
        self.last_actions = []
        self.last_temps = []
        self.last_outdoor = []
        self.last_rewards = []
        self.last_comfort_details = []  # 舒适度详情
        self.last_energy_details = []  # 能耗详情
        self.last_action_counts = Counter()
        
        # 训练历史（用于保存）
        self.training_history = {
            "episodes": [],
            "rewards": [],
            "temps": [],
            "comfort_ratios": [],
            "energy_consumption": [],
            "eval_episodes": [],
            "eval_rewards": [],
        }

    def log_episode_curves(self, actions, temps, outdoors, rewards, comfort_details=None, energy_details=None):
        self.last_actions = actions
        self.last_temps = temps
        self.last_outdoor = outdoors
        self.last_rewards = rewards
        self.last_comfort_details = comfort_details or []
        self.last_energy_details = energy_details or []
        self.last_action_counts = Counter(actions)
    
    def save_episode_data(self, episode, actions, temps, outdoors, rewards, comfort_details, energy_details):
        """保存单个episode的详细数据"""
        episode_data = {
            "episode": episode,
            "steps": len(actions),
            "actions": actions,
            "temperatures": temps,
            "outdoor_temps": outdoors,
            "rewards": rewards,
            "comfort_details": comfort_details,
            "energy_details": energy_details,
            "avg_temp": float(np.mean(temps)) if temps else 0,
            "comfort_ratio": float(np.mean([1 if 20 <= t <= 24 else 0 for t in temps])) if temps else 0,
            "total_energy": float(np.sum(energy_details)) if energy_details else 0,
        }
        self.episode_data.append(episode_data)
        
        # 保存到JSON文件
        json_path = os.path.join(self.save_dir, f"episode_{episode}.json")
        with open(json_path, 'w') as f:
            json.dump(episode_data, f, indent=2)
    
    def save_training_summary(self):
        """保存训练总结"""
        summary = {
            "total_episodes": len(self.train_rewards),
            "training_history": self.training_history,
            "final_stats": {
                "avg_reward": float(np.mean(self.train_rewards[-50:])) if len(self.train_rewards) >= 50 else float(np.mean(self.train_rewards)) if self.train_rewards else 0,
                "avg_temp": float(np.mean(self.train_temps[-50:])) if len(self.train_temps) >= 50 else float(np.mean(self.train_temps)) if self.train_temps else 0,
                "avg_comfort_ratio": float(np.mean(self.train_comfort_ratios[-50:])) if len(self.train_comfort_ratios) >= 50 else float(np.mean(self.train_comfort_ratios)) if self.train_comfort_ratios else 0,
            }
        }
        
        summary_path = os.path.join(self.save_dir, "training_summary.json")
        with open(summary_path, 'w') as f:
            json.dump(summary, f, indent=2)
        
        # 保存CSV格式（便于分析）
        csv_path = os.path.join(self.save_dir, "training_history.csv")
        with open(csv_path, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['Episode', 'Reward', 'AvgTemp', 'ComfortRatio', 'EnergyConsumption'])
            for i in range(len(self.train_rewards)):
                writer.writerow([
                    i + 1,
                    self.train_rewards[i],
                    self.train_temps[i],
                    self.train_comfort_ratios[i] if i < len(self.train_comfort_ratios) else 0,
                    self.train_energy_consumption[i] if i < len(self.train_energy_consumption) else 0,
                ])
        
        log(f"💾 训练数据已保存到 {self.save_dir}")

    def plot(self, save_path=None):
        """增强版训练过程可视化（与 DQN 一致）"""
        fig, axes = plt.subplots(2, 3, figsize=(18, 10))

        # 1. 训练奖励
        ax = axes[0, 0]
        episodes = list(range(1, len(self.train_rewards) + 1))
        ax.plot(episodes, self.train_rewards, 'b-', alpha=0.4, label='Raw')
        if len(self.train_rewards) >= 10:
            # MA(10): 需要10个数据点，从第10个episode开始
            ma = np.convolve(self.train_rewards, np.ones(10) / 10, mode='valid')
            ma_episodes = list(range(10, len(self.train_rewards) + 1))  # 对齐到整数episode
            ax.plot(ma_episodes, ma, 'r-', linewidth=2, label='MA(10)')
        ax.set_title('Training Reward', fontsize=12, fontweight='bold')
        ax.set_xlabel('Episode')
        ax.set_ylabel('Reward')
        ax.legend()
        ax.grid(True, alpha=0.3)

        # 2. 平均温度
        ax = axes[0, 1]
        ax.plot(self.train_temps, 'g-', linewidth=1.5)
        ax.axhspan(20, 24, alpha=0.2, color='green', label='Comfort Zone')
        ax.axhline(22, color='r', linestyle='--', linewidth=1.5, label='Target 22°C')
        ax.set_title('Average Temperature', fontsize=12, fontweight='bold')
        ax.set_xlabel('Episode')
        ax.set_ylabel('Temperature (°C)')
        ax.legend()
        ax.grid(True, alpha=0.3)

        # 3. 舒适区时间比例
        ax = axes[0, 2]
        if self.train_comfort_ratios:
            ax.plot(self.train_comfort_ratios, 'purple', linewidth=1.5)
            ax.axhline(0.8, color='orange', linestyle='--', label='Target 80%')
        ax.set_title('Comfort Zone Ratio', fontsize=12, fontweight='bold')
        ax.set_xlabel('Episode')
        ax.set_ylabel('Ratio (0-1)')
        ax.set_ylim(0, 1.05)
        ax.legend()
        ax.grid(True, alpha=0.3)

        # 4. 能耗趋势
        ax = axes[1, 0]
        if self.train_energy_consumption:
            ax.plot(self.train_energy_consumption, 'orange', linewidth=1.5)
        ax.set_title('Energy Consumption', fontsize=12, fontweight='bold')
        ax.set_xlabel('Episode')
        ax.set_ylabel('Total Energy (kW)')
        ax.grid(True, alpha=0.3)

        # 5. 评估奖励
        ax = axes[1, 1]
        if self.eval_rewards:
            ax.plot(self.eval_episodes, self.eval_rewards, 'ro-', markersize=6, linewidth=2)
        ax.set_title('Evaluation Reward', fontsize=12, fontweight='bold')
        ax.set_xlabel('Episode')
        ax.set_ylabel('Eval Reward')
        ax.grid(True, alpha=0.3)

        # 6. 温度分布（最近50个episode）
        ax = axes[1, 2]
        if len(self.train_temps) >= 50:
            recent_temps = self.train_temps[-50:]
            ax.hist(recent_temps, bins=20, color='skyblue', edgecolor='black', alpha=0.7)
            ax.axvline(22, color='r', linestyle='--', linewidth=2, label='Target 22°C')
            ax.axvspan(20, 24, alpha=0.2, color='green', label='Comfort Zone')
        ax.set_title('Temperature Distribution (Last 50 Episodes)', fontsize=12, fontweight='bold')
        ax.set_xlabel('Temperature (°C)')
        ax.set_ylabel('Frequency')
        ax.legend()
        ax.grid(True, alpha=0.3, axis='y')

        plt.tight_layout()
        if save_path:
            plt.savefig(save_path, dpi=150, bbox_inches='tight')
            log(f"📊 图表已保存: {save_path}")
        plt.show()

    def plot_episode_curves(self, episode, save_dir=None):
        if not self.last_actions:
            return
        steps = range(1, len(self.last_actions) + 1)

        decoded = [action_to_values(a) for a in self.last_actions]
        fan_seq = [d['fan'] for d in decoded]
        supply_seq = [K2C(d['supply_temp']) for d in decoded]
        heat_seq = [K2C(d['heat_setpoint']) for d in decoded]
        cool_seq = [K2C(d['cool_setpoint']) for d in decoded]

        # 增强版：4行3列，添加舒适度和能耗（与 DQN 一致）
        fig, axes = plt.subplots(4, 3, figsize=(20, 16))

        ax = axes[0, 0]
        ax.plot(steps, self.last_temps, label='Room Temp', linewidth=2, color='blue')
        ax.plot(steps, self.last_outdoor, label='Outdoor Temp', alpha=0.6, color='gray', linestyle='--')
        ax.axhspan(20, 24, alpha=0.15, color='green', label='Comfort Zone 20-24°C')
        ax.axhline(22, color='r', linestyle='--', linewidth=1.5, label='Target 22°C')
        ax.set_title(f'Episode {episode} | Temperature Profile', fontsize=11, fontweight='bold')
        ax.set_xlabel('Step (15min)')
        ax.set_ylabel('Temperature (°C)')
        ax.legend(loc='best')
        ax.grid(True, alpha=0.3)

        ax = axes[0, 1]
        ax.plot(steps, self.last_rewards, 'tab:orange', linewidth=1.5)
        ax.set_title('Step Rewards', fontsize=11, fontweight='bold')
        ax.set_xlabel('Step')
        ax.set_ylabel('Reward')
        ax.axhline(0, color='gray', linestyle='--', alpha=0.5)
        ax.grid(True, alpha=0.3)

        # 舒适度详情
        ax = axes[0, 2]
        if self.last_comfort_details:
            ax.plot(steps, self.last_comfort_details, 'green', linewidth=1.5, label='Comfort Reward')
            ax.axhline(0, color='gray', linestyle='--', alpha=0.5)
            # 标记舒适区
            comfort_zones = [1 if 20 <= t <= 24 else 0 for t in self.last_temps]
            ax2 = ax.twinx()
            ax2.fill_between(steps, 0, comfort_zones, alpha=0.2, color='green', label='In Comfort Zone')
            ax2.set_ylabel('Comfort Zone (0/1)', color='green')
            ax2.set_ylim(-0.1, 1.1)
        ax.set_title('Comfort Reward Details', fontsize=11, fontweight='bold')
        ax.set_xlabel('Step')
        ax.set_ylabel('Comfort Reward')
        ax.legend(loc='upper left')
        ax.grid(True, alpha=0.3)

        ax = axes[1, 0]
        ax.scatter(steps, self.last_actions, s=15, alpha=0.6, color='steelblue')
        ax.set_title('Action Index per Step', fontsize=11, fontweight='bold')
        ax.set_xlabel('Step')
        ax.set_ylabel('Action Index (0-23)')
        ax.set_ylim(-1, max(self.last_actions) + 2)
        ax.grid(True, alpha=0.3)

        ax = axes[1, 1]
        items = sorted(self.last_action_counts.items(), key=lambda x: x[0])
        if items:
            labels, counts = zip(*items)
            ax.bar(labels, counts, color='steelblue', edgecolor='black', alpha=0.7)
        ax.set_title('Action Usage Histogram', fontsize=11, fontweight='bold')
        ax.set_xlabel('Action Index')
        ax.set_ylabel('Usage Count')
        ax.grid(True, axis='y', alpha=0.3)

        ax = axes[2, 0]
        ax.plot(steps, supply_seq, label='Supply Temp', linewidth=1.5, color='red')
        ax.plot(steps, heat_seq, label='Heat Setpoint', linewidth=1.5, color='orange')
        ax.plot(steps, cool_seq, label='Cool Setpoint', linewidth=1.5, color='blue')
        ax.set_title('Supply Temperature & Setpoints', fontsize=11, fontweight='bold')
        ax.set_xlabel('Step')
        ax.set_ylabel('Temperature (°C)')
        ax.legend()
        ax.grid(True, alpha=0.3)

        ax = axes[2, 1]
        ax.plot(steps, fan_seq, label='Fan u', color='tab:purple', linewidth=1.5)
        ax.set_title('Fan Control', fontsize=11, fontweight='bold')
        ax.set_xlabel('Step')
        ax.set_ylabel('0-1')
        ax.set_ylim(0, 1.05)
        ax.legend()
        ax.grid(True, alpha=0.3)

        # 能耗详情
        ax = axes[2, 2]
        if self.last_energy_details:
            ax.plot(steps, self.last_energy_details, 'orange', linewidth=1.5, label='Energy Cost')
            ax.set_title('Energy Consumption per Step', fontsize=11, fontweight='bold')
            ax.set_xlabel('Step')
            ax.set_ylabel('Energy (kW)')
            ax.legend()
            ax.grid(True, alpha=0.3)
        else:
            ax.axis('off')

        # 温度分布直方图
        ax = axes[3, 0]
        if self.last_temps:
            ax.hist(self.last_temps, bins=20, color='skyblue', edgecolor='black', alpha=0.7)
            ax.axvline(22, color='r', linestyle='--', linewidth=2, label='Target 22°C')
            ax.axvspan(20, 24, alpha=0.2, color='green', label='Comfort Zone')
            ax.set_title('Temperature Distribution', fontsize=11, fontweight='bold')
            ax.set_xlabel('Temperature (°C)')
            ax.set_ylabel('Frequency')
            ax.legend()
            ax.grid(True, alpha=0.3, axis='y')

        # 奖励分解（舒适度 vs 能耗）
        ax = axes[3, 1]
        if self.last_comfort_details and self.last_energy_details:
            # 累积奖励
            cum_comfort = np.cumsum(self.last_comfort_details)
            cum_energy = np.cumsum([-e for e in self.last_energy_details])  # 负值表示惩罚
            ax.plot(steps, cum_comfort, 'g-', linewidth=2, label='Cumulative Comfort')
            ax.plot(steps, cum_energy, 'orange', linewidth=2, label='Cumulative Energy (neg)')
            ax.axhline(0, color='gray', linestyle='--', alpha=0.5)
            ax.set_title('Cumulative Reward Components', fontsize=11, fontweight='bold')
            ax.set_xlabel('Step')
            ax.set_ylabel('Cumulative Reward')
            ax.legend()
            ax.grid(True, alpha=0.3)
        else:
            ax.axis('off')

        # 统计信息
        ax = axes[3, 2]
        ax.axis('off')
        avg_temp = np.mean(self.last_temps) if self.last_temps else 0
        comfort_ratio = np.mean([1 if 20 <= t <= 24 else 0 for t in self.last_temps]) if self.last_temps else 0
        total_energy = sum(self.last_energy_details) if self.last_energy_details else 0
        total_reward = sum(self.last_rewards) if self.last_rewards else 0
        avg_comfort_reward = np.mean(self.last_comfort_details) if self.last_comfort_details else 0
        
        stats_text = f"""Episode {episode} Statistics
{'='*50}
Total Reward: {total_reward:.2f}
Avg Temperature: {avg_temp:.2f}°C
Comfort Zone Ratio: {comfort_ratio*100:.1f}%
Total Energy: {total_energy:.2f} kW
Avg Comfort Reward: {avg_comfort_reward:.2f}
Steps: {len(self.last_actions)}
Unique Actions: {len(self.last_action_counts)}
"""
        ax.text(0.1, 0.5, stats_text, fontsize=10, family='monospace',
                verticalalignment='center', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.3))

        plt.tight_layout()
        if save_dir:
            os.makedirs(save_dir, exist_ok=True)
            fig.savefig(os.path.join(save_dir, f"ppo_curves_ep{episode}.png"), dpi=150)
        plt.show()

# =====================================================================
# 训练函数
# =====================================================================

def train():
    print("=" * 70, flush=True)
    print("🚀 BOPTEST PPO 训练 (强化版)", flush=True)
    print("=" * 70, flush=True)

    # 检查连接
    log("检查BOPTEST连接...")
    try:
        resp = requests.get(f"{BOPTEST_URL}/testcases", timeout=10)
        log("✅ 连接成功!")
    except Exception as e:
        log(f"❌ 连接失败: {e}", "ERROR")
        log(f"请确认BOPTEST_URL是否正确: {BOPTEST_URL}", "ERROR")
        return None, None

    env = BOPTESTEnv()
    agent = PPOAgent(state_dim=env.obs_dim, action_dim=env.action_dim)
    os.makedirs(DATA_SAVE_DIR, exist_ok=True)
    os.makedirs(PLOT_SAVE_DIR, exist_ok=True)
    monitor = Monitor(save_dir=DATA_SAVE_DIR)

    print(f"\n配置:", flush=True)
    print(f"  Episodes: {TOTAL_EPISODES}", flush=True)
    print(f"  步数/Episode: {STEPS_PER_EPISODE}", flush=True)
    print(f"  动作空间: {NUM_ACTIONS}", flush=True)
    print(f"  熵系数: {ENTROPY_COEF} (控制探索)", flush=True)
    print("=" * 70, flush=True)

    best_reward = float('-inf')

    for episode in range(1, TOTAL_EPISODES + 1):
        print(f"\n{'=' * 60}", flush=True)
        print(f"📊 Episode {episode}/{TOTAL_EPISODES}", flush=True)
        print(f"{'=' * 60}", flush=True)

        rollout = {
            'states': [],
            'actions': [],
            'rewards': [],
            'dones': [],
            'log_probs': [],
            'values': [],
            'last_value': 0.0,
        }

        try:
            state = env.reset()
        except Exception as e:
            log(f"❌ Reset失败: {e}", "ERROR")
            env.stop()
            env.testid = None
            time.sleep(2)
            continue

        total_reward = 0
        temps = []
        action_counts = {}
        actions_seq = []
        rewards_seq = []
        outdoor_seq = []
        comfort_details = []  # 收集舒适度详情
        energy_details = []  # 收集能耗详情

        # 预热步骤：固定高风+高供风+高暖设+高冷设，帮助快速升温（与 DQN 保持一致）
        WARMUP_STEPS = 4
        WARMUP_ACTION = 23  # Fan=0.9, TSup=23°C, Heat=23°C, Cool=28°C
        for _ in range(WARMUP_STEPS):
            # 使用 WARMUP_ACTION，但需要获取对应的 log_prob 和 value
            with torch.no_grad():
                state_t = torch.FloatTensor(state).unsqueeze(0)
                logits = agent.model.actor(state_t)
                probs = torch.softmax(logits, dim=-1)
                dist = torch.distributions.Categorical(probs)
                log_prob = dist.log_prob(torch.tensor([WARMUP_ACTION]))
                value = agent.model.critic(state_t).squeeze(-1)
            
            next_state, reward, done, info = env.step(WARMUP_ACTION)
            
            rollout['states'].append(state)
            rollout['actions'].append(WARMUP_ACTION)
            rollout['rewards'].append(reward)
            rollout['dones'].append(float(done))
            rollout['log_probs'].append(log_prob.item())
            rollout['values'].append(value.item())
            
            state = next_state
            total_reward += reward
            temps.append(info['room_temp'])
            actions_seq.append(WARMUP_ACTION)
            rewards_seq.append(reward)
            outdoor_seq.append(info['outdoor_temp'])
            action_counts[WARMUP_ACTION] = action_counts.get(WARMUP_ACTION, 0) + 1
            # 收集详细信息
            rd = info['reward_detail']
            comfort_details.append(rd['comfort'])
            energy_details.append(abs(rd['energy']))  # 能耗绝对值
            if done:
                break

        for step in range(STEPS_PER_EPISODE):
            action, log_prob, value, probs = agent.select_action(
                state, deterministic=False
            )

            action_counts[action] = action_counts.get(action, 0) + 1

            next_state, reward, done, info = env.step(action)

            rollout['states'].append(state)
            rollout['actions'].append(action)
            rollout['rewards'].append(reward)
            rollout['dones'].append(float(done))
            rollout['log_probs'].append(log_prob)
            rollout['values'].append(value)

            total_reward += reward
            temps.append(info['room_temp'])
            actions_seq.append(action)
            rewards_seq.append(reward)
            outdoor_seq.append(info['outdoor_temp'])
            
            # 收集详细信息
            rd = info['reward_detail']
            comfort_details.append(rd['comfort'])
            energy_details.append(abs(rd['energy']))  # 能耗绝对值

            if (step + 1) % 10 == 0:
                top3_actions = np.argsort(probs)[-3:][::-1]
                top3_str = ", ".join(
                    [f"A{a}:{probs[a]:.2f}" for a in top3_actions]
                )

                print(
                    f"  Step {step+1:3d} | "
                    f"室温={info['room_temp']:5.1f}°C | "
                    f"室外={info['outdoor_temp']:5.1f}°C | "
                    f"R={reward:+6.2f} | "
                    f"动作={action:2d} | "
                    f"{action_to_string(action)}",
                    flush=True,
                )
                print(f"           Top3概率: [{top3_str}]", flush=True)
                rd = info['reward_detail']
                print(
                    f"           细分: comfort={rd['comfort']:+.2f}, energy={rd['energy']:+.2f}, smooth={rd['smooth']:+.2f}",
                    flush=True,
                )

            state = next_state
            if done:
                break

        # bootstrap 最后价值（若未超时且非终止，可用当前状态价值）
        with torch.no_grad():
            last_value = agent.model.get_value(
                torch.FloatTensor(state).unsqueeze(0)
            ).item()
        rollout['last_value'] = last_value

        loss_info = agent.update(rollout)

        avg_temp = np.mean(temps) if temps else 0
        comfort_ratio = np.mean([1 if 20 <= t <= 24 else 0 for t in temps]) if temps else 0
        total_energy = np.sum(energy_details) if energy_details else 0
        
        monitor.train_rewards.append(total_reward)
        monitor.train_temps.append(avg_temp)
        monitor.train_comfort_ratios.append(comfort_ratio)
        monitor.train_energy_consumption.append(total_energy)
        monitor.entropies.append(loss_info['entropy'])
        
        # 更新训练历史
        monitor.training_history["episodes"].append(episode)
        monitor.training_history["rewards"].append(float(total_reward))
        monitor.training_history["temps"].append(float(avg_temp))
        monitor.training_history["comfort_ratios"].append(float(comfort_ratio))
        monitor.training_history["energy_consumption"].append(float(total_energy))
        
        # 记录episode曲线
        monitor.log_episode_curves(actions_seq, temps, outdoor_seq, rewards_seq, 
                                   comfort_details, energy_details)
        
        # 保存episode详细数据
        monitor.save_episode_data(episode, actions_seq, temps, outdoor_seq, 
                                 rewards_seq, comfort_details, energy_details)

        unique_actions = len(action_counts)
        most_common = sorted(action_counts.items(), key=lambda x: -x[1])[:3]

        print(f"\n✅ Episode {episode} 完成", flush=True)
        print(f"   总奖励: {total_reward:.1f}", flush=True)
        print(f"   平均室温: {avg_temp:.1f}°C (目标 20-24°C)", flush=True)
        print(f"   舒适区时间比例: {comfort_ratio*100:.1f}%", flush=True)
        print(f"   总能耗: {total_energy:.2f} kW", flush=True)
        print(
            f"   Loss: {loss_info['loss']:.4f} | Entropy: {loss_info['entropy']:.4f}",
            flush=True,
        )
        print(f"   使用了 {unique_actions} 种不同动作", flush=True)
        print(
            f"   最常用动作: {[(a, action_to_string(a), c) for a, c in most_common]}",
            flush=True,
        )

        env.stop()
        
        # 每10个episode保存一次训练总结（轻量级）
        if episode % 10 == 0:
            monitor.save_training_summary()

        if episode % EVAL_FREQUENCY == 0:
            print(f"\n🔍 评估中 (确定性策略)...", flush=True)
            eval_rewards = []

            for _ in range(2):
                state = env.reset()
                ep_reward = 0
                for _ in range(STEPS_PER_EPISODE):
                    action, _, _, _ = agent.select_action(
                        state, deterministic=True
                    )
                    state, reward, done, _ = env.step(action)
                    ep_reward += reward
                    if done:
                        break
                eval_rewards.append(ep_reward)
                env.stop()

            avg_eval = np.mean(eval_rewards) if eval_rewards else 0
            monitor.eval_episodes.append(episode)
            monitor.eval_rewards.append(avg_eval)

            print(f"📊 评估奖励: {avg_eval:.1f}", flush=True)

            if avg_eval > best_reward:
                best_reward = avg_eval
                agent.save("best_model.pth")
                log(f"💾 保存最佳模型 best_model.pth | eval={avg_eval:.2f}")

            # 保存并显示训练图表
            plot_path = os.path.join(PLOT_SAVE_DIR, f"training_progress_ep{episode}.png")
            monitor.plot(save_path=plot_path)
            monitor.plot_episode_curves(episode, PLOT_SAVE_DIR)
            
            # 保存训练总结
            monitor.save_training_summary()
            
            print(
                f"\n{'='*60}",
                flush=True,
            )
            print(
                f"Episode {episode}: 训练={total_reward:.1f}, 评估={avg_eval:.1f}, Entropy={loss_info['entropy']:.3f}",
                flush=True,
            )
            print(
                f"{'='*60}\n",
                flush=True,
            )

        time.sleep(0.2)

    # 保存最终模型
    agent.save("final_model.pth")
    log("训练结束，已保存 final_model.pth")
    
    # 保存最终训练总结
    monitor.save_training_summary()
    
    # 生成最终图表
    final_plot_path = os.path.join(PLOT_SAVE_DIR, "training_final.png")
    monitor.plot(save_path=final_plot_path)
    monitor.plot_episode_curves("final", PLOT_SAVE_DIR)
    
    log(f"📊 所有训练数据已保存到: {monitor.save_dir}")
    log(f"📈 所有图表已保存到: {PLOT_SAVE_DIR}")

    print("\n" + "=" * 70, flush=True)
    print("🎉 训练完成!", flush=True)
    print(f"   最佳评估奖励: {best_reward:.1f}", flush=True)
    print("=" * 70, flush=True)

    return agent, monitor

# =====================================================================
# 启动提示
# =====================================================================

print("=" * 60)
print("📌 PPO训练代码已加载 (强化版)")
print("=" * 60)
print(f"\nBOPTEST URL: {BOPTEST_URL}")
print(f"动作空间: {NUM_ACTIONS} 个动作")
print(f"熵系数: {ENTROPY_COEF} (越大探索越多)")
print("\n运行: train()")
print("=" * 60)


📌 PPO训练代码已加载 (强化版)

BOPTEST URL: http://localhost
动作空间: 24 个动作
熵系数: 0.05 (越大探索越多)

运行: train()


In [4]:
train()

🚀 BOPTEST PPO 训练 (强化版)
[15:04:28] INFO  | 检查BOPTEST连接...
[15:04:28] INFO  | ✅ 连接成功!
[15:04:28] INFO  | PPO Agent 初始化: 24 actions
[15:04:28] INFO  |   Actor学习率: 0.0001
[15:04:28] INFO  |   Critic学习率: 0.0005
[15:04:28] INFO  |   熵系数: 0.05
[15:04:28] INFO  |   PPO clip: 0.2

配置:
  Episodes: 500
  步数/Episode: 672
  动作空间: 24
  熵系数: 0.05 (控制探索)

📊 Episode 1/500
[15:04:28] DEBUG | 可用测试案例: [{'testcaseid': 'bestest_air'}, {'testcaseid': 'bestest_hydronic'}, {'testcaseid': 'bestest_hydronic_heat_pump'}, {'testcaseid': 'multizone_office_complex_air'}, {'testcaseid': 'multizone_office_simple_air'}, {'testcaseid': 'multizone_office_simple_hydronic'}, {'testcaseid': 'multizone_residential_hydronic'}, {'testcaseid': 'singlezone_commercial_hydronic'}, {'testcaseid': 'testcase2'}, {'testcaseid': 'testcase3'}, {'testcaseid': 'twozone_apartment_hydronic'}]
[15:04:28] INFO  | 选择测试案例: bestest_air
[15:04:58] ERROR | ❌ 选择失败: HTTPConnectionPool(host='localhost', port=80): Read timed out. (read timeout=30)
[15

KeyboardInterrupt: 